# A2C Training 

This notebook trains an A2C (Advantage Actor-Critic) agent on the Daladala environment with 12 hyperparameter configurations.

## Section 1: Install and Import Dependencies

In [ ]:
!pip install stable-baselines3 gymnasium torch pandas numpy opencv-python --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 7.0 MB/s eta 0:00:00


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import StopTrainingOnNoModelImprovement
import warnings
warnings.filterwarnings('ignore')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


## Section 2: Mount Google Drive (for saving models and results)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create output directories
os.makedirs('/content/drive/MyDrive/daladala_results/models/a2c', exist_ok=True)
os.makedirs('/content/drive/MyDrive/daladala_results/results', exist_ok=True)
print("✓ Google Drive mounted successfully")

Mounted at /content/drive
✓ Google Drive mounted successfully


## Section 3: Define the DaladalaEnv Environment

In [ ]:
class DaladalaEnv(gym.Env):
    """Daladala (mini-bus) optimization environment with 5 actions and 14 observations."""
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 12}

    def __init__(self, render_mode=None):
        super().__init__()
        self.observation_space = spaces.Box(-1, 1, shape=(14,), dtype=np.float32)
        self.action_space = spaces.Discrete(5)  # 0:Move, 1:Pickup, 2:Dropoff, 3:Stop, 4:SpeedUp
        self.render_mode = render_mode

        # Fixed route Ubungo → Posta (right then up)
        self.route = [(x, 14) for x in range(15)] + [(14, y) for y in range(13, -1, -1)]
        self.high_demand_stops = [(4,14), (8,14), (14,8), (14,3)]

        # These will be randomized each reset
        self.police_checkpoints = []
        self.traffic_lights = []
        self.traffic_light_states = {}  # Stores state (Red=1, Green=0) for each light

        # Pools for randomization
        self.available_positions = [pos for pos in self.route if pos not in self.high_demand_stops]

        self.max_steps = 350
        self.physical_max = 50
        self.light_cycle = 0  # Track light cycle deterministically

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.step_count = 0
        self.passengers = 0
        self.money = 0.0
        self.pos_idx = 0
        self.speed = 0
        self.fined = False
        self.light_cycle = 0

        # Randomize hazard positions each episode
        available = [pos for pos in self.route if pos not in self.high_demand_stops]
        if len(available) >= 7:  # 3 police + 4 traffic lights
            sampled = np.random.choice(len(available), 7, replace=False)
            self.police_checkpoints = [available[i] for i in sampled[:3]]
            self.traffic_lights = [available[i] for i in sampled[3:7]]

            # Assign random constant state (Red=1, Green=0) for each light this episode
            # This ensures the light stays the same throughout the episode
            self.traffic_light_states = {pos: np.random.randint(0, 2) for pos in self.traffic_lights}

        # Initialize deterministic passenger counts per stop (seeded by position)
        self.passengers_at_stop = {}
        for stop in self.high_demand_stops:
            # Deterministic: same stop always has same initial count
            seed_val = hash(stop) % 11  # 0-10 passengers
            self.passengers_at_stop[stop] = seed_val

        return self._get_obs(), {}

    def _get_obs(self):
        """
        Generate observation based on current location and environment state.
        Observations are DETERMINISTIC per location to match training/visualization.
        """
        if self.pos_idx >= len(self.route):
            x, y = 14, 0
        else:
            x, y = self.route[self.pos_idx]

        # === CURRENT LOCATION HAZARDS ===
        # Traffic light: Constant state for this episode (Red=1, Green=0)
        light_is_red = self.traffic_light_states.get((x, y), 0)

        # Police checkpoint detection
        police_here = 1 if (x, y) in self.police_checkpoints else 0

        # Must stop at THIS location?
        must_stop_now = 1 if (light_is_red or police_here) else 0

        # === NEXT LOCATION HAZARDS ===
        next_idx = min(self.pos_idx + 1, len(self.route) - 1)
        next_x, next_y = self.route[next_idx]

        # Check what's ahead
        next_light_is_red = self.traffic_light_states.get((next_x, next_y), 0)
        police_ahead = 1 if (next_x, next_y) in self.police_checkpoints else 0
        must_stop_next = 1 if (next_light_is_red or police_ahead) else 0

        # === PASSENGER STATE ===
        # At high-demand stop: passengers waiting (deterministic)
        at_stop = 1 if (x, y) in self.high_demand_stops else 0
        passengers_waiting = self.passengers_at_stop.get((x, y), 0) if at_stop else 0

        # === DISTANCE AHEAD (for lookahead) ===
        # Distance to next traffic light (in next 5 cells)
        dist_to_light = 5
        for i in range(self.pos_idx + 1, min(self.pos_idx + 6, len(self.route))):
            if self.route[i] in self.traffic_lights:
                dist_to_light = i - self.pos_idx
                break

        # Distance to next police (in next 5 cells)
        dist_to_police = 5
        for i in range(self.pos_idx + 1, min(self.pos_idx + 6, len(self.route))):
            if self.route[i] in self.police_checkpoints:
                dist_to_police = i - self.pos_idx
                break

        # === BUILD OBSERVATION VECTOR (all normalized to [-1, 1]) ===
        obs = np.array([
            x / 14.0 * 2 - 1,                      # [0] position_x
            y / 14.0 * 2 - 1,                      # [1] position_y
            self.passengers / self.physical_max * 2 - 1,  # [2] current_passengers
            self.money / 150000.0 * 2 - 1,        # [3] money_earned
            self.speed / 3.0 * 2 - 1,              # [4] current_speed
            light_is_red * 2 - 1,                  # [5] light_is_red_HERE (critical)
            police_here * 2 - 1,                   # [6] police_checkpoint_HERE
            must_stop_now * 2 - 1,                 # [7] must_stop_now_HERE (critical)
            at_stop * 2 - 1,                       # [8] at_high_demand_stop
            passengers_waiting / 10.0 * 2 - 1,    # [9] passengers_waiting_at_stop
            must_stop_next * 2 - 1,                # [10] must_stop_next_location
            dist_to_light / 5.0 * 2 - 1,          # [11] distance_to_traffic_light
            dist_to_police / 5.0 * 2 - 1,         # [12] distance_to_police
            self.step_count / self.max_steps * 2 - 1,  # [13] episode_progress
        ], dtype=np.float32)

        return obs

    def step(self, action):
        """
        Action: 0=Move, 1=Pickup, 2=Dropoff, 3=Stop, 4=SpeedUp
        Movement is ALWAYS automatic. Actions are overlaid on movement.
        Rewards guide agent toward optimal actions based on current state.
        """
        self.step_count += 1
        self.light_cycle += 1  # Update traffic light cycle

        terminated = truncated = False
        x, y = self.route[self.pos_idx]

        # === PHASE 1: AUTOMATIC MOVEMENT (always happens) ===
        if self.pos_idx < len(self.route) - 1:
            self.pos_idx += 1
        else:
            terminated = True

        # === PHASE 2: EXECUTE ACTION ===
        reward = 0.0

        # Observe the CURRENT location (before action)
        light_is_red = self.traffic_light_states.get((x, y), 0)
        police_here = 1 if (x, y) in self.police_checkpoints else 0
        must_stop_here = 1 if (light_is_red or police_here) else 0
        at_stop = 1 if (x, y) in self.high_demand_stops else 0

        # === INTELLIGENT REWARD SYSTEM ===
        # We know the "right" action for each state, so rewards guide strongly

        if action == 0:  # MOVE action (advance to next cell)
            # Movement already happened automatically
            # This action is mostly for consistency; reward small progress bonus
            reward += 2

            # PENALTY: Moved through hazard without stopping
            if must_stop_here:
                reward -= 40  # Heavy penalty: ran through red light or police checkpoint

        elif action == 1:  # PICKUP action
            if at_stop and self.passengers < self.physical_max:
                # GOOD: Picking up at a stop
                base_add = max(3, self.passengers_at_stop.get((x, y), 0))
                add = min(base_add, self.physical_max - self.passengers)
                self.passengers += add
                reward += 15  # High reward for correct action

                # Deduct waiting passengers
                if (x, y) in self.passengers_at_stop:
                    self.passengers_at_stop[(x, y)] = max(0, self.passengers_at_stop[(x, y)] - add)
            else:
                # BAD: Picked up when not at stop
                reward -= 5

            # PENALTY: Picking up at hazard zone
            if must_stop_here:
                reward -= 10

        elif action == 2:  # DROPOFF action
            if at_stop and self.passengers > 0:
                # GOOD: Dropping off at a stop
                drop = min(self.passengers, max(3, self.passengers // 2 + 1))
                self.passengers -= drop
                self.money += drop * 1000
                reward += 12  # Good reward for revenue
            else:
                # BAD: Dropped off when not at stop
                reward -= 8

            # PENALTY: Dropping off at hazard zone
            if must_stop_here:
                reward -= 10

        elif action == 3:  # STOP action (slows down / waits)
            self.speed = max(0, self.speed - 1)

            # GOOD: Stopped at hazard location
            if must_stop_here:
                reward += 25  # Strong reward: correct safety action
            else:
                # BAD: Unnecessary stop
                reward -= 3

        elif action == 4:  # SPEEDUP action
            # GOOD: Speeding up in safe zones
            if not must_stop_here and self.passengers <= 40:
                self.speed = min(self.speed + 1, 3)
                reward += 3
            else:
                # BAD: Speeding in danger or when overloaded
                if must_stop_here:
                    reward -= 15  # Dangerous
                if self.passengers > 40:
                    reward -= 30  # Could crash
                    terminated = True  # Crash!

        # === PHASE 3: SAFETY VIOLATIONS ===
        # Check destination after automatic movement
        new_x, new_y = self.route[self.pos_idx] if self.pos_idx < len(self.route) else (14, 0)

        # Police checkpoint consequences
        if (new_x, new_y) in self.police_checkpoints:
            if self.passengers > 40:
                reward -= 50  # Severe: overloaded at police
                self.fined = True
                terminated = True
            elif self.passengers > 33:
                reward -= 20  # Violation: illegal capacity
                self.fined = True

        # === PHASE 4: PROGRESS & COMPLETION ===
        # Base movement reward (small, to encourage progress)
        reward += 1

        # Destination completion bonus
        if terminated:
            reward += 100  # Large bonus for reaching destination
            if self.passengers <= 33 and not self.fined:
                reward += 50  # Bonus for legal completion

        # === PHASE 5: STATE UPDATES ===
        truncated = self.step_count >= self.max_steps

        return self._get_obs(), reward, terminated, truncated, {}

    def render(self):
        pass  # Rendering disabled for Colab

print("✓ DaladalaEnv class defined successfully")

✓ DaladalaEnv class defined successfully


## Section 4: Define A2C Hyperparameter Configurations

In [ ]:
# 12 A2C hyperparameter configurations for systematic tuning
a2c_configs = [
    {"name": "LR_1e4_n5_g099", "learning_rate": 1e-3, "n_steps": 5, "gamma": 0.65, "gae_lambda": 0.95, "ent_coef": 0.0},
    {"name": "LR_1e4_n5_g0995", "learning_rate": 1e-4, "n_steps": 5, "gamma": 0.70, "gae_lambda": 0.95, "ent_coef": 0.0},
    {"name": "LR_3e4_n8_g099", "learning_rate": 3e-4, "n_steps": 8, "gamma": 0.75, "gae_lambda": 0.95, "ent_coef": 0.005},
    {"name": "LR_3e4_n8_g0995", "learning_rate": 3e-4, "n_steps": 8, "gamma": 0.77, "gae_lambda": 0.95, "ent_coef": 0.005},
    {"name": "LR_5e4_n10_g099", "learning_rate": 5e-4, "n_steps": 10, "gamma": 0.80, "gae_lambda": 1.0, "ent_coef": 0.01},
    {"name": "LR_5e4_n10_g0995", "learning_rate": 5e-4, "n_steps": 10, "gamma": 0.85, "gae_lambda": 1.0, "ent_coef": 0.01},
    {"name": "LR_7e4_n5_g099", "learning_rate": 7e-4, "n_steps": 5, "gamma": 0.90, "gae_lambda": 0.95, "ent_coef": 0.05},
    {"name": "LR_7e4_n5_g0995", "learning_rate": 7e-4, "n_steps": 5, "gamma": 0.95, "gae_lambda": 0.95, "ent_coef": 0.05},
    {"name": "LR_1e3_n8_g099", "learning_rate": 1e-3, "n_steps": 8, "gamma": 0.995, "gae_lambda": 1.0, "ent_coef": 0.01},
    {"name": "LR_1e3_n8_g0995", "learning_rate": 1e-3, "n_steps": 8, "gamma": 0.67, "gae_lambda": 1.0, "ent_coef": 0.01},
    {"name": "LR_1e3_n10_g099", "learning_rate": 1e-3, "n_steps": 10, "gamma": 0.77, "gae_lambda": 0.95, "ent_coef": 0.0},
    {"name": "LR_1e3_n10_g0995", "learning_rate": 1e-3, "n_steps": 10, "gamma": 0.98, "gae_lambda": 0.95, "ent_coef": 0.0},
]

print(f"✓ {len(a2c_configs)} A2C configurations defined")

✓ 12 A2C configurations defined


## Section 5: Train A2C with All Configurations

In [ ]:
results = {}
best_reward = -float('inf')
best_config = None
best_model = None

total_configs = len(a2c_configs)

for idx, config in enumerate(a2c_configs, 1):
    print(f"\n{'='*60}")
    print(f"Training Configuration {idx}/{total_configs}: {config['name']}")
    print(f"{'='*60}")

    # Create environment
    env = DaladalaEnv()

    # Initialize A2C model with configuration
    model = A2C(
        'MlpPolicy',
        env,
        learning_rate=config['learning_rate'],
        n_steps=config['n_steps'],
        gamma=config['gamma'],
        gae_lambda=config['gae_lambda'],
        ent_coef=config['ent_coef'],
        verbose=1, # Changed from 0 to 1 for progress updates
        device='auto' # Changed from 'cpu' to 'auto' to use GPU if available
    )

    # Train for 300,000 timesteps
    print(f"Training for 300,000 timesteps...")
    model.learn(total_timesteps=300000)

    # Evaluate model on 50 episodes
    print(f"Evaluating on 50 episodes...")
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=50, deterministic=True)

    results[config['name']] = {
        'config': config,
        'mean_reward': mean_reward,
        'std_reward': std_reward
    }

    print(f"✓ Mean Reward: {mean_reward:.2f} (±{std_reward:.2f})")

    # Track best model
    if mean_reward > best_reward:
        best_reward = mean_reward
        best_config = config['name']
        best_model = model
        print(f"★ New best model!")

    env.close()

print(f"\n{'='*60}")
print(f"TRAINING COMPLETE")
print(f"{'='*60}")
print(f"Best Configuration: {best_config}")
print(f"Best Mean Reward: {best_reward:.2f}")

Streaming output truncated to the last 5000 lines.
|    time_elapsed       | 12        |
|    total_timesteps    | 7000      |
| train/                |           |
|    entropy_loss       | -0.408    |
|    explained_variance | -4.21e-05 |
|    learning_rate      | 0.001     |
|    n_updates          | 699       |
|    policy_loss        | 12.1      |
|    value_loss         | 1.16e+04  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 29        |
|    ep_rew_mean        | 351       |
| time/                 |           |
|    fps                | 574       |
|    iterations         | 800       |
|    time_elapsed       | 13        |
|    total_timesteps    | 8000      |
| train/                |           |
|    entropy_loss       | -0.229    |
|    explained_variance | -0.000288 |
|    learning_rate      | 0.001     |
|    n_updates          | 799       |
|    policy_loss        | 0.41      |

## Section 6: Save Best Model and Results

In [ ]:
# Save best model
best_model_path = '/content/drive/MyDrive/daladala_results/models/a2c/best_a2c'
best_model.save(best_model_path)
print(f"✓ Best model saved to: {best_model_path}")

# Save results as JSON
results_json_path = '/content/drive/MyDrive/daladala_results/results/a2c_results.json'
results_summary = {}
for config_name, config_results in results.items():
    results_summary[config_name] = {
        'mean_reward': float(config_results['mean_reward']),
        'std_reward': float(config_results['std_reward']),
        'hyperparameters': {
            'learning_rate': config_results['config']['learning_rate'],
            'n_steps': config_results['config']['n_steps'],
            'gamma': config_results['config']['gamma'],
            'gae_lambda': config_results['config']['gae_lambda'],
            'ent_coef': config_results['config']['ent_coef']
        }
    }

with open(results_json_path, 'w') as f:
    json.dump(results_summary, f, indent=2)
print(f"✓ Results saved to: {results_json_path}")

# Display results table
print("\n" + "="*80)
print("RESULTS SUMMARY - All 12 A2C Configurations")
print("="*80)
results_df = pd.DataFrame([
    {
        'Config': name,
        'Mean Reward': f"{results[name]['mean_reward']:.2f}",
        'Std Reward': f"{results[name]['std_reward']:.2f}",
        'LR': results[name]['config']['learning_rate'],
        'n_steps': results[name]['config']['n_steps'],
        'gamma': results[name]['config']['gamma']
    }
    for name in results.keys()
])
print(results_df.to_string(index=False))
print("="*80)

✓ Best model saved to: /content/drive/MyDrive/daladala_results/models/a2c/best_a2c
✓ Results saved to: /content/drive/MyDrive/daladala_results/results/a2c_results.json

RESULTS SUMMARY - All 12 A2C Configurations
          Config Mean Reward Std Reward     LR  n_steps  gamma
  LR_1e4_n5_g099      426.20      20.75 0.0010        5  0.650
 LR_1e4_n5_g0995      427.52      19.85 0.0001        5  0.700
  LR_3e4_n8_g099      423.56      21.33 0.0003        8  0.750
 LR_3e4_n8_g0995      423.56      18.92 0.0003        8  0.770
 LR_5e4_n10_g099      417.08      23.76 0.0005       10  0.800
LR_5e4_n10_g0995      428.84      22.56 0.0005       10  0.850
  LR_7e4_n5_g099      421.36      22.71 0.0007        5  0.900
 LR_7e4_n5_g0995      404.52      22.01 0.0007        5  0.950
  LR_1e3_n8_g099      375.12      21.98 0.0010        8  0.995
 LR_1e3_n8_g0995      420.48      20.34 0.0010        8  0.670
 LR_1e3_n10_g099      378.64      23.13 0.0010       10  0.770
LR_1e3_n10_g0995      429.72   

## Section 7: Test Best Model on Sample Episodes

In [ ]:
# Test the best model on 5 sample episodes
print(f"\nTesting best model ({best_config}) on 5 sample episodes:")
print("="*60)

env = DaladalaEnv()
episode_rewards = []

for ep in range(5):
    obs, _ = env.reset()
    total_reward = 0
    done = False
    steps = 0

    while not done:
        action, _ = best_model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        done = terminated or truncated
        steps += 1

    episode_rewards.append(total_reward)
    print(f"Episode {ep+1}: Reward = {total_reward:.2f} (Steps: {steps})")

env.close()

print("="*60)
print(f"Sample Episodes Mean Reward: {np.mean(episode_rewards):.2f}")
print(f"Sample Episodes Std Reward:  {np.std(episode_rewards):.2f}")
print("\n✓ A2C training and evaluation complete!")
print(f"✓ Models and results saved to Google Drive")


Testing best model (LR_1e3_n10_g0995) on 5 sample episodes:
Episode 1: Reward = 424.00 (Steps: 29)
Episode 2: Reward = 424.00 (Steps: 29)
Episode 3: Reward = 402.00 (Steps: 29)
Episode 4: Reward = 424.00 (Steps: 29)
Episode 5: Reward = 446.00 (Steps: 29)
Sample Episodes Mean Reward: 424.00
Sample Episodes Std Reward:  13.91

✓ A2C training and evaluation complete!
✓ Models and results saved to Google Drive
